In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_html(url):
    r = requests.get(url)
    return r.text
def get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    pages = soup.find('div', class_='search-result-paging').find_all('a')
    total_pages = pages[-2].get('href').split('/')[-2]
    return int(total_pages)
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    ads = soup.find('div', class_='main-wrap__content').find_all('div', class_='company-item')
    spisok = []
    for ad in ads:
        
        try:
            title = ad.find('div', class_='company-item__title').find('a').text.strip()
        except:
            title = ''
        try:
            url = 'https://www.rusprofile.ru'+str(ad.find('div', class_='company-item__title').find('a').get('href'))
        except:
            url = ''
        try:
            address = ad.find('address').text.strip()
        except:
            address = ''
        ad_subs = ad.find_all('div', class_='company-item-info')
        try:
            manager = ad_subs[0].find('dt').text
        except:
            manager = ''
        try:
            manager_name = ad_subs[0].find('dd').text
        except:
            manager_name = ''
        try:
            INN = ad_subs[1].find('dt').text
        except:
            INN = ''
        try:
            INN_value = ad_subs[1].find('dd').text
        except:
            INN_value = ''
        try:
            dealing = ad_subs[2].find('dt').text
        except:
            dealing = ''
        try:
            okved = ad_subs[2].find('dd').text
        except:
            okved = ''
        

        data = {'Название компании': title, 'URL-адрес': url,
               'Адрес': address, 'Руководитель': manager,
                'ФИО руководителя': manager_name,
                'Идентификатор': INN, 'ИНН': INN_value,
                'Вид деятельности': dealing, 'ОКВЭД': okved}
        spisok.append(data)
        
        
    return spisok

In [3]:
def main():
    url = 'https://www.rusprofile.ru/codes/683210/moskva'
    base_url = 'https://www.rusprofile.ru/codes/683210/moskva/'
    total_pages = get_total_pages(get_html(url))
    spisok = []
    for i in range(3, total_pages-1):
        url_gen = base_url + str(i)
        html = get_html(url_gen)
        spisok.extend(get_page_data(html))
    return spisok

In [4]:
okved_683210_moskva = pd.DataFrame(main())
#okved_683210_moskva.to_excel('okved_683210_moskva.xlsx')

In [26]:
okved_683210_moskva = pd.read_excel('okved_683210_moskva.xlsx')

In [33]:
urls = okved_683210_moskva['URL-адрес']

In [34]:
import time
spisok = []
for url in urls:
    time.sleep(2)
    html = get_html(url)
    soup = BeautifulSoup(html, 'lxml')
    try:
        ads = ''
        ads = soup.find_all('div', class_='change-item')
        if len(ads) > 1:
            for ad in ads:
                date_change = ''
                text_change = ''
                date_change = ad.find('div', class_='change-item__date').text
                text_change = ad.find('div', class_='change-item__text').text.strip()
                data = {'URL-адрес': url, 'Дата изменений': date_change, 'Текст изменений': text_change}
                spisok.append(data)
    except:
        pass
changes_in_okved_683210_moskva = pd.DataFrame(spisok)
changes_in_okved_683210_moskva.to_excel('changes_in_okved_683210_moskva.xlsx')

# Читаю по парсингу на https://habr.com/ru/company/ods/blog/346632/

In [11]:
page_link = 'http://knowyourmeme.com/memes/all/page/1'
#page_link = 'https://www.rusprofile.ru/codes/683210/moskva'

response = requests.get(page_link)
response

<Response [403]>

In [12]:
for key, value in response.request.headers.items():
    print(key+": "+value)

User-Agent: python-requests/2.22.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive


In [8]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36'

In [13]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [28]:
html = response.text
html[:1000]

'<!DOCTYPE html>\n<html xmlns:fb=\'https://www.facebook.com/2008/fbml\' xmlns=\'https://www.w3.org/1999/xhtml\'>\n<head>\n<meta content=\'text/html; charset=utf-8\' http-equiv=\'Content-Type\'>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":115,"agent":""}</script>\n<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("e

In [15]:
soup = BeautifulSoup(html,'html.parser')

In [19]:
soup.html.head.title.text

'All Entries | Know Your Meme'

In [21]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="/memes/8-mile-rap-battle"><img alt="8 Mile Rap Battle" data-src="https://i.kym-cdn.com/entries/icons/medium/000/031/011/cover4.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="8 Mile Rap Battle"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>

In [22]:
obj.attrs['href']

'/memes/8-mile-rap-battle'

In [25]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
meme_links[:3]

[<a class="photo" href="/memes/8-mile-rap-battle"><img alt="8 Mile Rap Battle" data-src="https://i.kym-cdn.com/entries/icons/medium/000/031/011/cover4.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="8 Mile Rap Battle"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/jacob-rees-mogg-lying-down-in-the-commons"><img alt="Jacob Rees-Mogg Lying Down In The Commons" data-src="https://i.kym-cdn.com/entries/icons/medium/000/031/010/Screenshot_2019-09-04_at_1.11.07_PM.png" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Jacob Rees-Mogg Lying Down In The Commons"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/spirituality-definition-the-true-inner-meaning"><img alt="spirituality definition the true inner meaning" data-src="https://i.kym-cdn.com/entries/icon

In [26]:
meme_links = [link.attrs['href'] for link in meme_links]
meme_links[:10]

['/memes/8-mile-rap-battle',
 '/memes/jacob-rees-mogg-lying-down-in-the-commons',
 '/memes/spirituality-definition-the-true-inner-meaning',
 '/memes/thomas-had-never-seen-such-bullshit-before',
 '/memes/geedis-and-the-land-of-ta',
 '/memes/sites/zao',
 '/memes/events/forever-21-bankruptcy',
 '/memes/subcultures/spirit-halloween',
 '/memes/mike-wazowski-sulley-face-swap',
 '/memes/depression-blue-body-scan']

In [ ]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы

        page_number: int/string
            номер страницы для парсинга

    """
    # составляем ссылку на страницу поиска
    page_link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page_number)

    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['http://knowyourmeme.com' + link.attrs['href'] for link in meme_links]

    return meme_links

In [30]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 178.18.212.84


In [34]:
!pip3 install PySocks

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [36]:
import PySocks
import socket
PySocks.set_default_proxy(PySocks.SOCKS5, "localhost", 9150)
socket.socket = PySocks.socksocket

ModuleNotFoundError: No module named 'PySocks'

In [37]:
import PySocks

ModuleNotFoundError: No module named 'PySocks'